In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd

bgg_Category = pd.read_csv('scraped/bgg_Category.csv')
bgg_GameFamily = pd.read_csv('scraped/bgg_GameFamily.csv')
bgg_GameItem = pd.read_csv('scraped/bgg_GameItem.csv')
bgg_GameType = pd.read_csv('scraped/bgg_GameType.csv')
bgg_Mechanic = pd.read_csv('scraped/bgg_Mechanic.csv')
bgg_Person = pd.read_csv('scraped/bgg_Person.csv')
bgg_Publisher = pd.read_csv('scraped/bgg_Publisher.csv')

In [2]:
drop=[
'bga_id',
'dbpedia_id',
'luding_id',
'spielen_id',
'wikidata_id',
'wikipedia_id']

In [3]:
predictor = [
#'name',
'year',
'bgg_id',
'designer',
]
response_vars = [
'num_votes',
'avg_rating',
#'stddev_rating',
#'rank',
'bayes_rating',
"novelty_norm",
]
actor_covariates = [
#"is.man"
]
game_covariates = [
 'complexity',
 'game_type',
 'min_players',
 'max_players',
 'category',
# 'mechanic',
# 'publisher',
# 'min_players_rec',
# 'max_players_rec',
# 'min_players_best',
# 'max_players_best',
# 'min_age',
# 'min_age_rec',
# 'min_time',
# 'max_time',
# 'cooperative',
# 'family',
# 'language_dependency',
# 'artist',
# # filter out ?
# 'compilation',
# 'compilation_of',
# 'implementation',
# 'integration',
#
]

<h1>Cleaning</h1>

In [4]:
#df_core = bgg_GameItem.copy()[core+game_covariates+response]
df_core = bgg_GameItem.drop(columns=drop).copy()
core = df_core[df_core.avg_rating.notna()] # at least 1 ratings
core = core[core.bayes_rating.notna()] # at least 30 ratings
core = core[(core.year >= 1990) & (core.year < 2024)] # only games after key:  1990 <= year < 2024
core.designer = core.designer.str.split(",")
core [core.designer.map(lambda x: "3" in x, na_action="ignore").fillna(False)] = pd.NA # 3 is designer="uncredited"
core = core[core.designer.notna()] # must include designer
core = core[core.designer.map(len) <= 16] # huge groups are rare and lead to effects
core = core[core.mechanic.notna()] # must include mechanics
core.mechanic = core.mechanic.str.split(",").map(set).map(sorted).map(tuple)
core = core.sort_values("year").reset_index(drop=True)
core.head(2)

,bgg_id,name,year,game_type,designer,artist,publisher,min_players,max_players,min_players_rec,...,family,implementation,integration,rank,num_votes,avg_rating,stddev_rating,bayes_rating,complexity,language_dependency
0,18330.0,Dragon Noir 1: L'Exil,1990.0,4664,[80],"4911,11802,11992",116,2.0,7.0,2.0,...,"7005,3273",NaN,NaN,13062.0,54.0,6.95648,1.14795,5.53777,2.5000,NaN
1,2379.0,Guesstures,1990.0,5498,"[8083, 11050]",NaN,"3115,858,51,20,28,4653",4.0,99.0,4.0,...,"10714,6042",NaN,NaN,6188.0,1702.0,5.90289,1.53140,5.66313,1.1802,NaN


In [5]:
"""by_id = core.sort_values("year").reset_index(drop=True)
by_id.designer = by_id.designer.str.split(",")
by_id = by_id.explode("designer")
by_id["event_type"] = "event"
main = ["bgg_id","year","designer","event_type"]
by_id = by_id[main+[x for x in by_id if x not in main]]
by_id.head(2)"""

'by_id = core.sort_values("year").reset_index(drop=True)\nby_id.designer = by_id.designer.str.split(",")\nby_id = by_id.explode("designer")\nby_id["event_type"] = "event"\nmain = ["bgg_id","year","designer","event_type"]\nby_id = by_id[main+[x for x in by_id if x not in main]]\nby_id.head(2)'

<h1>Compute Novelties

In [6]:
#step 1
configurations = core.mechanic.unique()
lut = {v:k for k,v in enumerate(configurations)}
dummies = pd.Series(configurations).explode().str.get_dummies().groupby(level=0).sum()
dummies

,2001,2002,2003,2004,2005,2007,2008,2009,2010,2011,...,3007,3096,3097,3098,3099,3100,3101,3102,3103,3104
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9452,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9453,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9455,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#step 2: euclidean distances between each pair of configurations
from sklearn.metrics import pairwise_distances
dist_matrix = pd.DataFrame(pairwise_distances(dummies),index=dummies.index,columns=dummies.index)
dist_matrix = dist_matrix/dist_matrix.max(None)
dist_matrix.head(2)

,0,1,2,3,4,5,6,7,8,9,...,9447,9448,9449,9450,9451,9452,9453,9454,9455,9456
0,0.000000,0.361158,0.442326,0.442326,0.294884,0.294884,0.361158,0.255377,0.361158,0.361158,...,0.294884,0.361158,0.442326,0.510754,0.417029,0.361158,0.390095,0.442326,0.390095,0.417029
1,0.361158,0.000000,0.489010,0.531610,0.361158,0.361158,0.466252,0.329690,0.417029,0.417029,...,0.361158,0.417029,0.489010,0.551677,0.466252,0.417029,0.442326,0.442326,0.442326,0.466252


In [8]:
#step 3: novelty = average of distances of the focal game to all of the games that existed at that time
from tqdm import tqdm
import numpy as np

def min_max(df):
    return (df-df.min(None))/(df.max(None)-df.min(None))

mechanics_by_year = core[["bgg_id","year","mechanic"]].copy().reset_index(drop=True)
mechanics_by_year["configuration_id"] = mechanics_by_year.mechanic.map(lambda x :lut[x])
mechanics_by_year = mechanics_by_year.set_index("year").sort_index()
novelty = mechanics_by_year.reset_index()
for i,ix in enumerate(mechanics_by_year.index):
    focal = novelty.iloc[i]["configuration_id"]
    previous = mechanics_by_year.loc[:ix,"configuration_id"]
    novelty.loc[i,"novelty"] = dist_matrix.to_numpy()[previous.values,focal].mean()
novelty["novelty_norm"] = novelty[["novelty"]].apply(min_max).convert_dtypes()
core_vars = core.merge(novelty,on=["year","bgg_id"])

<h1> Add dummy events


In [9]:
# actor_dummy: who could participate in events actor dummies
dummy_actor = core.explode(column="designer").groupby("designer")["year"].min().reset_index()
dummy_actor["event_type"]= "add_actor"
dummy_actor.head(2)

,designer,year,event_type
0,1,1990.0,add_actor
1,10,1996.0,add_actor


In [10]:
# dummy events for setting gender on actors
person_gendered = pd.read_csv('bgg_person_gendered.csv')
person_gendered.bgg_id = person_gendered.bgg_id.astype(str)
dummy_gender = pd.merge(pd.Series(core.designer.explode().unique(),name="designer"),person_gendered,left_on="designer",right_on="bgg_id")
dummy_gender = dummy_gender.drop(columns=["bgg_id","name"])
dummy_gender = dummy_gender.rename({"is.man":"weight"},axis=1)
dummy_gender["event_type"]="add_gender"
dummy_gender.head(2)

,designer,weight,event_type
0,80,-1,add_gender
1,8083,1,add_gender


In [11]:
# actor_dummy: who could participate in events actor dummies
dummy_actor_by_year = core.explode(column="designer").groupby("designer")["year"].min().reset_index()
interval = np.arange(dummy_actor_by_year.year.min(),dummy_actor_by_year.year.max()+1)
dummy_tenure = dummy_actor_by_year.set_index("designer").apply(lambda x:interval-x.year,result_type="expand",axis=1).set_axis(interval,axis=1).T
dummy_tenure = dummy_tenure.stack().clip(lower=-1).reset_index().set_axis(["year","designer","weight"],axis=1).drop_duplicates(subset=["designer","weight"])#.dropna(subset="weight")#.replace(-1,pd.NA)
dummy_tenure["event_type"]="add_tenure"
dummy_tenure


,year,designer,weight,event_type
0,1990.0,1,0.0,add_tenure
1,1990.0,10,-1.0,add_tenure
2,1990.0,1000,-1.0,add_tenure
3,1990.0,100012,-1.0,add_tenure
4,1990.0,100025,-1.0,add_tenure
...,...,...,...,...
334317,2023.0,99909,5.0,add_tenure
334318,2023.0,99951,6.0,add_tenure
334319,2023.0,99957,6.0,add_tenure
334320,2023.0,99958,6.0,add_tenure


<h1> Export AND set outcome/target


In [12]:
import csv
for target in response_vars:
    outcome = core_vars.copy()
    outcome = core_vars.rename({target:"weight"},axis=1,errors="raise")
    cols = predictor  + game_covariates + actor_covariates + ["event_type","weight"]
    outcome["event_type"] = "event"
    eventlist = pd.concat([outcome,dummy_actor,dummy_gender,dummy_tenure],axis=0)
    eventlist = eventlist.sort_values(["year","event_type"],ascending=(True,True)).reset_index(drop=True).convert_dtypes()
    numeric = eventlist.select_dtypes("number").columns 
    eventlist.loc[:,numeric] = eventlist.loc[:,numeric].fillna(0)
    eventlist = eventlist.explode(column="designer")
    eventlist = eventlist.sort_values(["year","event_type"],ascending=(True,True)).reset_index(drop=True)
    eventlist[cols].to_csv("../RHOM_input/bgg_"+target+".csv",index=False,quoting=csv.QUOTE_NONNUMERIC)

In [15]:
core_vars.family


0                                      7005,3273
1                                     10714,6042
2                   14835,61979,72016,3228,43648
3                                            157
4                                 10643,70710,19
                          ...                   
19795                                72535,66113
19796               14835,81787,73596,5666,61977
19797    22184,5838,65328,25158,8374,73596,13265
19798                     22184,10634,8374,64227
19799               22184,7564,77906,72487,65774
Name: family, Length: 19800, dtype: object

In [18]:
import networkx

core_vars

,bgg_id,name,year,game_type,designer,artist,publisher,min_players,max_players,min_players_rec,...,num_votes,avg_rating,stddev_rating,bayes_rating,complexity,language_dependency,mechanic_y,configuration_id,novelty,novelty_norm
0,18330.0,Dragon Noir 1: L'Exil,1990.0,4664,[80],"4911,11802,11992",116,2.0,7.0,2.0,...,54.0,6.95648,1.14795,5.53777,2.5000,NaN,"(2011, 2026)",0,0.298267,0.105752
1,2379.0,Guesstures,1990.0,5498,"[8083, 11050]",NaN,"3115,858,51,20,28,4653",4.0,99.0,4.0,...,1702.0,5.90289,1.53140,5.66313,1.1802,NaN,"(2019, 2028, 2035, 2073)",1,0.375135,0.237155
2,12629.0,Marching Through Georgia,1990.0,4664,[65],3439,"3429,94",2.0,2.0,2.0,...,59.0,6.18305,1.68223,5.51791,2.8333,NaN,"(2046, 2070, 2072, 2855, 2863, 2947, 2967)",2,0.420546,0.314783
3,173270.0,Ogre / G.E.V.,1990.0,"4664,5496",[22],"20471,13114,35109,844",19,1.0,3.0,1.0,...,110.0,7.55564,1.54919,5.59262,2.0000,NaN,"(2015, 2026, 2070, 2072, 2676, 2822, 2855, 289...",3,0.441921,0.351324
4,23818.0,1847: Pfalz,1990.0,NaN,[7019],NaN,5429,3.0,5.0,3.0,...,54.0,7.26111,1.17830,5.54854,3.6000,NaN,"(2002, 2005)",4,0.321625,0.145683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19795,329484.0,Pyramido,2023.0,NaN,[111776],144973,"39502,23043,6194,54932,22560,38505,33626,23031...",2.0,4.0,2.0,...,158.0,7.36506,1.34144,5.61147,1.7778,NaN,"(2002, 2041, 2079, 3001)",9452,0.383789,0.251948
19796,183685.0,"Mr. President: The American Presidency, 2001-2020",2023.0,5496,[1587],24469,52,1.0,1.0,1.0,...,535.0,8.56675,1.95697,6.07047,4.2000,NaN,"(2001, 2070, 2072, 2080, 2819)",9453,0.391404,0.264966
19797,367150.0,Dune: War for Arrakis,2023.0,4664,"[934, 935]","24858,34328,10548,14348",21608,1.0,4.0,1.0,...,144.0,6.44924,3.58367,5.54340,3.2500,NaN,"(2019, 2046, 2070, 2072, 2080, 2850, 2856)",9454,0.444350,0.355476
19798,346255.0,Sushi Boat,2023.0,NaN,"[113380, 103305]",128797,16687,2.0,5.0,2.0,...,38.0,7.26579,1.64257,5.53220,2.0000,NaN,"(2004, 2047, 2082, 2987, 3005)",9455,0.410024,0.296797


In [33]:
pd.Series(bgg_GameFamily.name).str.split(":").map(lambda x:x[0]).value_counts()

name
Series                       1372
Game                         1026
Cities                        410
Theme                         187
History                       180
                             ... 
Mountain                        1
Neon Comet Games map sets       1
Sport                           1
Mechanic                        1
Games                           1
Name: count, Length: 71, dtype: int64

In [59]:
themes = bgg_GameFamily[bgg_GameFamily.name.str.contains("Theme")]#.name.str.split(":").map(lambda x:x[1]).unique()
themes

,bgg_id,name
333,5141,Theme: Cthulhu Mythos
356,5570,Video Game Theme: Honfoglaló
363,5601,Theme: Anime / Manga
370,5610,Theme: Steampunk
371,5611,Theme: Cyberpunk
...,...,...
4900,98795,Theme: Metal Smithing
4901,98973,Video Game Theme: Five Nights at Freddy's
4904,99005,Theme: Origami
4907,99080,Theme: Electricity Generation


In [53]:
bgg_Category.name.tolist()

['Political',
 'Card Game',
 'Nautical',
 'Abstract Strategy',
 'Fantasy',
 'Transportation',
 'Farming',
 'Civilization',
 'Science Fiction',
 'Dice',
 'Wargame',
 'Exploration',
 'Economic',
 'Adventure',
 'Bluffing',
 'Horror',
 'Word Game',
 'Negotiation',
 'Trivia',
 'Puzzle',
 'City Building',
 'Party Game',
 'Racing',
 'Action / Dexterity',
 'Mafia',
 'Trains',
 'Medieval',
 'Prehistoric',
 'Real-time',
 'Sports',
 'Deduction',
 'Murder/Mystery',
 "Children's Game",
 'Expansion for Base-game',
 'Collectible Components',
 'Memory',
 'Fighting',
 'Miniatures',
 'American Civil War',
 'World War II',
 'Ancient',
 'Napoleonic',
 'Arabian',
 'Music',
 'American West',
 'Maze',
 'Movies / TV / Radio theme',
 'World War I',
 'Modern Warfare',
 'Renaissance',
 'Electronic',
 'American Revolutionary War',
 'Humor',
 'Spies/Secret Agents',
 'Mythology',
 'Environmental',
 'Territory Building',
 'Industry / Manufacturing',
 'Animals',
 'Pirates',
 'Korean War',
 'Novel-based',
 'Educationa

In [57]:
core_vars[core_vars.category.isna()].sort_values("num_votes")

,bgg_id,name,year,game_type,designer,artist,publisher,min_players,max_players,min_players_rec,...,num_votes,avg_rating,stddev_rating,bayes_rating,complexity,language_dependency,mechanic_y,configuration_id,novelty,novelty_norm
111,11915.0,Tabaijana: Flucht von der Feuerinsel,1990.0,NaN,[7],81481,1043,2.0,5.0,2.0,...,30.0,6.06000,1.651590,5.50694,1.5000,NaN,"(2023,)",88,0.288873,0.089693
19431,376893.0,Cyberion,2023.0,NaN,[37729],37730,"51383,12047",1.0,2.0,1.0,...,30.0,8.23333,0.863456,5.53555,NaN,NaN,"(2040, 2819)",2334,0.313854,0.132398
17466,350015.0,Rumms: Schnipp die Krone,2021.0,NaN,"[70407, 16265]","143804,38196,11898",37,2.0,4.0,2.0,...,30.0,7.50578,1.249740,5.52908,1.0000,NaN,"(2072, 2860)",1297,0.312345,0.129817
18673,365876.0,Atlantic Robot League,2022.0,NaN,[146455],146456,221,2.0,5.0,2.0,...,31.0,6.41935,2.032390,5.50810,1.0000,NaN,"(2014,)",158,0.297159,0.103859
14525,297555.0,翡翠の商人 (Jade Merchant),2019.0,NaN,[98921],50254,35514,2.0,5.0,2.0,...,31.0,5.95161,1.340170,5.50730,1.0000,NaN,"(2012,)",110,0.288264,0.088653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9058,165986.0,Royals,2014.0,"5497,5499",[770],"14611,11825","29,17812,9068,50638",2.0,5.0,2.0,...,2692.0,7.17028,1.197290,6.45217,2.2136,NaN,"(2004, 2041, 2080)",3839,0.334468,0.167637
15688,242684.0,Reavers of Midgard,2019.0,5497,[97699],105971,"26427,36401,31839,27389,32591,2726",2.0,4.0,2.0,...,2795.0,7.38770,1.323430,6.57624,2.9362,NaN,"(2004, 2041, 2072, 2082, 2843)",6981,0.385447,0.254783
18437,367220.0,Sea Salt & Paper,2022.0,5499,"[1727, 72628]","147767,147768","19260,10768,48742,37681,46724,1391,17179,7992,...",2.0,4.0,2.0,...,3460.0,7.39688,1.379000,6.66731,1.4368,NaN,"(2004, 2040, 2661, 2981)",8575,0.368397,0.225636
4834,34585.0,Keltis,2008.0,5499,[2],"11968,11902","37,267,4335,2366,5657,4617,6214,8291,5597,22,9...",2.0,4.0,2.0,...,6359.0,6.42103,1.349950,6.18380,1.6421,1.025641,"(2004, 2040, 2078)",1607,0.324133,0.14997


In [60]:
bgg_GameFamily

,bgg_id,name
0,1,Admin: Test Family
1,2,Game: Carcassonne
2,3,Game: Catan
3,4,Series: The Chicken Family of Zoch
4,5,Game: Bohnanza
...,...,...
4921,99475,Series: Giant Edition (Spin Master)
4922,99576,Game: Broadside and Salvo (Long Face Games)
4923,99601,Series: Small Box Euros (Devir)
4924,99694,Series: Next Station (Blue Orange)


In [61]:
bgg_GameItem.columns.to_list()

['bgg_id',
 'name',
 'year',
 'game_type',
 'designer',
 'artist',
 'publisher',
 'min_players',
 'max_players',
 'min_players_rec',
 'max_players_rec',
 'min_players_best',
 'max_players_best',
 'min_age',
 'min_age_rec',
 'min_time',
 'max_time',
 'category',
 'mechanic',
 'cooperative',
 'compilation',
 'compilation_of',
 'family',
 'implementation',
 'integration',
 'rank',
 'num_votes',
 'avg_rating',
 'stddev_rating',
 'bayes_rating',
 'complexity',
 'language_dependency',
 'bga_id',
 'dbpedia_id',
 'luding_id',
 'spielen_id',
 'wikidata_id',
 'wikipedia_id']

In [76]:
themes.astype("category")

,bgg_id,name
333,5141,Theme: Cthulhu Mythos
356,5570,Video Game Theme: Honfoglaló
363,5601,Theme: Anime / Manga
370,5610,Theme: Steampunk
371,5611,Theme: Cyberpunk
...,...,...
4900,98795,Theme: Metal Smithing
4901,98973,Video Game Theme: Five Nights at Freddy's
4904,99005,Theme: Origami
4907,99080,Theme: Electricity Generation


In [85]:
games = core_vars.family.str.split(",").explode()
pd.merge(games.astype("str"),themes.astype("str"),left_on="family",right_on="bgg_id",how="left").dropna()

,family,bgg_id,name
25,69366,69366,Theme: Siege
45,5613,5613,Theme: Vikings
54,28718,28718,Theme: Music Making & Makers
57,13265,13265,Theme: Deserts
75,26464,26464,Theme: Alternate History
...,...,...,...
56537,60795,60795,Theme: Dreams / Nightmares
56538,65832,65832,Theme: Psychology
56551,66113,66113,Theme: Construction
56563,13265,13265,Theme: Deserts


In [79]:
games.astype("category")

0         7005
0         3273
1        10714
1         6042
2        14835
         ...  
19799    22184
19799     7564
19799    77906
19799    72487
19799    65774
Name: family, Length: 56573, dtype: category
Categories (3453, object): ['1', '10', '100', '10051', ..., '99694', '9974', '9976', '9983']

In [86]:
themes

,bgg_id,name
333,5141,Theme: Cthulhu Mythos
356,5570,Video Game Theme: Honfoglaló
363,5601,Theme: Anime / Manga
370,5610,Theme: Steampunk
371,5611,Theme: Cyberpunk
...,...,...
4900,98795,Theme: Metal Smithing
4901,98973,Video Game Theme: Five Nights at Freddy's
4904,99005,Theme: Origami
4907,99080,Theme: Electricity Generation


In [96]:
themes.name.str.contains("wars",case=False).sum()

0

In [98]:
bgg_Category[bgg_Category.name.str.contains("wars",case=False)]

,bgg_id,name
68,1108,American Indian Wars


In [132]:
games_types = bgg_GameItem.set_index("bgg_id")family.str.split(",").explode().reset_index().set_index("game_type")
games_types.join(bgg_GameType.astype("str").set_index("bgg_id")).reset_index().dropna(subset="name").name.value_counts()

name
War Game           10268
Abstract Game       4953
Children's Game     4158
Family Game         3224
Strategy Game       2941
Thematic            1640
Party Game           881
Customizable         576
Amiga                  1
Commodore 64           1
Arcade                 1
Atari ST               1
Name: count, dtype: int64

In [133]:
bgg_GameType.set_index("bgg_id")

,name
bgg_id,
4415,Amiga
4420,Commodore 64
4664,War Game
4665,Children's Game
4666,Abstract Game
4667,Customizable
4781,Arcade
4976,Atari ST
5496,Thematic


In [158]:
d = bgg_GameItem[["bgg_id","game_type","family"]]
d.family = d.family.str.split(",")
d.game_type = d.game_type.str.split(",")
d = d.explode("game_type").explode("mechanics")
d.iloc[:,-2:].value_counts()

/var/folders/t0/lg3jmzv573q7n1cv7kt891y40000gn/T/ipykernel_52433/1004530532.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d.family = d.family.str.split(",")
/var/folders/t0/lg3jmzv573q7n1cv7kt891y40000gn/T/ipykernel_52433/1004530532.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d.game_type = d.game_type.str.split(",")


game_type  family
4664       61979     4350
4666       61979     2628
4664       72016     1279
4666       26432      770
5497       8374       757
                     ... 
5496       43662        1
           43654        1
           43653        1
           43647        1
           28952        1
Name: count, Length: 8806, dtype: int64